###  Events information such as event name, event date and time with venue location are retrieved from eventbrite api

# Table of contents

### 1. Library
### 2. Setting up eventbrite api
### 3. Find all activities from api + including all pages
### 4. API URL builder
### 5. URL shortner
### 6. Searching events

### Reference

# 1. Library

In [1]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import re
import requests
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
from dateutil import tz
import numpy as np

## 2. Setting up eventbrite api

In [2]:
from eventbrite import Eventbrite
eventbrite = Eventbrite('api-key')
user = eventbrite.get_user()  # Not passing an argument returns yourself
apiKey='api-key'
print(user['id'])

print(user['name'])

333703320445
vikram patil


## 3. Find all activities from api + including all pages

In [9]:
# Below method converts the date and time from UTC to Austalia/Melbourne time

def convert_date(date):
    ################################
    # parent method = find_events()
    # input = UTC date   
    #output = Melboure date and time
    ################################
    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('Australia/Melbourne')
    date=str(date)

    utc = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    central = utc.astimezone(to_zone)
    date=central.date()
    time=central.time()
    return pd.Series([date,time])



# Below method find the required events information such as event lcoation, name, image and etc
def find_events(url,event_cat,event_name):
    ################################
    # parent method = find_activities()
    # input:
        # url - eventbrite specific url
        # event_cat - outdoor,indoor,special needs, hobbies
        # event_name - 18 events are present
        
    #output = all event details are fetched
    ################################
    total_data=pd.DataFrame()
    
    
    response=requests.get(url)
    resp_json_payload2 = response.json()

    start_page = resp_json_payload2['pagination']['page_number']
    last_page = resp_json_payload2['pagination']['page_count']

    ##################################
    # If api has mroe than one page results
    if resp_json_payload2['pagination']['page_count'] >1 :

        for i in range(1,last_page+1):
            new_url=url+'&page={}'.format(i)
            response=requests.get(new_url)
            resp_json = response.json()
            data=json_normalize(resp_json['events'])
            #col=['category_id','name.text','summary','is_free','start.utc','end.utc','status','url']
            col=['name.text','is_free','start.utc','end.utc','status','url','venue.address.latitude',\
                 'venue.address.longitude','venue.address.localized_address_display','venue.address.postal_code',\
                 'venue.address.city','logo.original.url','logo.original.height','logo.original.width','venue.name']
            data=data[col]
            total_data=total_data.append(data,ignore_index=True)

    # if api has single page result
    else:
        new_url=url
        response=requests.get(new_url)
        resp_json = response.json()
        data=json_normalize(resp_json['events'])
        #col=['category_id','name.text','summary','is_free','start.utc','end.utc','status','url']
        col=['name.text','is_free','start.utc','end.utc','status','url','venue.address.latitude',\
                 'venue.address.longitude','venue.address.localized_address_display','venue.address.postal_code',\
                 'venue.address.city','logo.original.url','logo.original.height','logo.original.width','venue.name']
        data=data[col]
        total_data=total_data.append(data,ignore_index=True)

    total_data['event_sub_cat']=event_name
    total_data['event_cat']=event_cat
    
    
    total_data['start.utc']=pd.to_datetime(total_data['start.utc'])
    total_data[['start_date','start_time']]=total_data.apply(lambda x:convert_date(x['start.utc']) ,axis=1)
    
    total_data['end.utc']=pd.to_datetime(total_data['end.utc'])
    total_data[['end_date','end_time']]=total_data.apply(lambda x:convert_date(x['end.utc']) ,axis=1)

    
    
    return total_data

### 4. API url builder

In [25]:
def find_activities(event_name_list,event_cat,km):
     ################################
    # input:
        # event_name_list - this is sub category of events to be searched
        # event_cat - this contains the main category of events such as - outdoor, indoor, hobbies and special needs
        # km - 100 km coverage area
        
    #output = dataframe of final result is sent
    ################################
    total_df = pd.DataFrame()
    for each_event in event_name_list:
        url = ('https://www.eventbriteapi.com/v3/events/search/?location.address={}&expand=venue&token={}&location.within={}&q={}'\
               .format('Melbourne',apiKey,km,each_event))

        df=find_events(url,event_cat,each_event)
        df.rename(columns={'name.text':'event_short_des','venue.address.latitude':'venue_lat',\
                           'venue.address.longitude':'venue_lon',\
                             'venue.address.localized_address_display':'venue_address','venue.address.postal_code':'postcode',\
                             'venue.address.city':'suburb','url':'event_url','logo.original.url':'event_image_url',\
                           'logo.original.height':'event_img_original_height',\
                           'logo.original.width':'event_img_original_width',
                           'venue.name':'venue_name'},inplace=True)
        df=df[['postcode','suburb','event_cat','event_sub_cat','event_short_des','is_free','start_date','start_time','end_date',\
                         'end_time','status','event_url','venue_lat','venue_lon','venue_name','venue_address','event_image_url',\
              'event_img_original_height','event_img_original_width']]
        total_df=total_df.append(df)
    
    total_df['event_img_original_width'].fillna(0, inplace=True)
    total_df['event_img_original_width']=total_df.event_img_original_width.astype(int)
    total_df['event_img_original_height'].fillna(0, inplace=True)
    total_df['event_img_original_height']=total_df.event_img_original_height.astype(int)
    total_df['nationality']='Australia'
    total_df['event_show_map']='True'
    total_df['event_show_map_link']='True'

    total_df.reset_index(drop=True,inplace=True)
    
    return total_df

### 5. URL shortner

In [1]:
# this method accepts long url and shortens it using tinyurl

import pyshorteners
from pyshorteners import Shorteners

def short_url(url):
    s = pyshorteners.Shortener(Shorteners.TINYURL, timeout=9000)
    try:
        short_url = s.short(url)
        shorturl = '[img src="{}"]'.format(short_url)
    except:
        #print('ERROR: {}'.format(url))
        shorturl=np.nan
    return shorturl

## 6. Searching events

In [27]:
event_name_list=['Seniors Festival','Cafe Walks','Park Walks','Networking','Tour','Spirituality','Healthy and Active Session']
event_cat='Outdoor'
km='100km'


outdoor_df = find_activities(event_name_list,event_cat,km)

In [32]:
outdoor_df.shape

(2593, 22)

In [31]:
event_name_list=['Seniors at Home','Computers for Seniors','yoga for Seniors','Tai Chi for Seniors','Book Club Seniors',\
                'Comedy Show']
event_cat='Indoor'
km='100km'


indoor_df = find_activities(event_name_list,event_cat,km)

In [33]:
indoor_df.shape

(135, 22)

In [35]:
event_name_list=['Art therapy for Seniors','Music for Seniors','Table Tennis','Vertical Gardens','Seniors Week',\
                'Cooking Classes','Pottery Classes','Poetry Classes','Trivia']
event_cat='Hobbies'
km='100km'


hobbies_df = find_activities(event_name_list,event_cat,km)
hobbies_df.shape

(224, 22)

In [36]:
event_name_list=['Dementia','Sensory Activities',\
                 'Craft Club']
event_cat='Special Needs'
km='100km'


special_df = find_activities(event_name_list,event_cat,km)
special_df.shape

(70, 22)

In [44]:
final_df = pd.concat([outdoor_df,indoor_df,hobbies_df,special_df])
final_df=final_df[~final_df['suburb'].isnull()]
final_df=final_df[~final_df['postcode'].isnull()]
final_df.reset_index(drop=True,inplace=True)

final_df.shape

(2825, 22)

In [45]:
final_df['is_free'] = final_df['is_free'] .map({True: 'Free', False: 'Charged'})
final_df['event_image_url']=final_df.apply(lambda x:short_url(x['event_image_url']),axis=1)
event_df=final_df[~final_df['event_image_url'].isnull()]
event_df['event_sub_cat']=event_df['event_sub_cat']+','+event_df['suburb']
event_df['data_type']='eventbrite'
event_df.to_csv('final_eventbrite.csv',index=False)

# Reference

1. https://github.com/ellisonleao/pyshorteners/issues/117
